# Caracterização de eletrofáceis por meio de inteligência artificial com abordagem supervisionada
## Treinamento, teste e análise do modelo

- **Nome do Arquivo**: treinamento_modelo.ipynb
- **Autor**: Rafael Takeguma Goto
- **Versão**: 1.0
- **Data de Criação**: 24/06/2024
- **Data de Modificação**: 24/06/2024

### Descrição: 
Este notebook contém o a etapa do trabalho referente à realização do treinamento, teste e análise do algoritmo de identificação de eletrofáceis a partir dos dados rotulados.

### Leitura do conjunto de dados

O conjunto de dados é constituído por arquivos csv gerados a partir da fusão da extração dos arquivos dlis com os arquivos agp de cada poço.

In [12]:
import glob
import pandas as pd

leituras = []      # Armazena as leituras dos arquivos .csv
nomes = []          # Armazena os nomes obtidos das leituras

for file in glob.glob(r'**/Dados-filtrados' + "/*.csv", recursive=True):
    try:
        # Salva os dados da leitura
        leitura = pd.read_csv(f'{file}')
        leituras.append(leitura)

        # Salva o nome do poço
        nomes.append(file[16:21])
    except:
        print(file)

### Divisão do conjunto de dados
A divisão dos dados que serão utilizados pelo algoritmo é feita com base no método *holdout*, do qual divide os dados em proporção de p para o treinamento do algoritmo e (1-p) para o teste do algoritmo. Foi definido p como 2/3, 66.67% dos dados para o treinamento e 33.34% para teste, como geralmente é feito (FACELI et al. 2011).